In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 10.9 MB/s 


# Preprocessing

In [3]:
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt
# import utils

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [4]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
#  Loading the database
url = "/content/drive/MyDrive/zomato.csv"
df = pd.read_csv(url)
print(df.head)

In [6]:
df=df.drop(['dish_liked'],axis=1)
df=df.drop(['url', 'address', 'phone', 'menu_item'], axis=1)
df=df.dropna()
df=df.drop_duplicates()

In [7]:
df.rate=df.rate.replace({'-':'0/5','NEW':'0/5'})
df.rate=df.rate.astype(str)
df.rate=df.rate.apply(lambda row: row.replace('/5',''))
df.rate=df.rate.astype(float)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
df['approx_cost(for two people)']=df['approx_cost(for two people)'].astype(str)
df['approx_cost(for two people)']=df['approx_cost(for two people)'].apply(lambda row: row.replace(',',''))
df['approx_cost(for two people)']=df['approx_cost(for two people)'].astype(float)

<ipython-input-8-6e01825ff696>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['approx_cost(for two people)']=df['approx_cost(for two people)'].astype(str)
<ipython-input-8-6e01825ff696>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['approx_cost(for two people)']=df['approx_cost(for two people)'].apply(lambda row: row.replace(',',''))
<ipython-input-8-6e01825ff696>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [9]:
df['rest_type'] = df['rest_type'].astype('str')
df['rest_type'] = df['rest_type'].apply(lambda string: string.split(','))
df['rest_type'] = df['rest_type'].apply(lambda lst: sorted([string.strip() for string in lst]))
df['rest_type'] = df['rest_type'].apply(lambda lst: ', '.join(lst))

# correct cuisines column (change order)
df['cuisines'] = df['cuisines'].astype('str')
df['cuisines'] = df['cuisines'].apply(lambda string: string.split(','))
df['cuisines'] = df['cuisines'].apply(lambda lst: sorted([string.strip() for string in lst]))
df['cuisines'] = df['cuisines'].apply(lambda lst: ', '.join(lst))

<ipython-input-9-28f9b57c6941>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rest_type'] = df['rest_type'].astype('str')
<ipython-input-9-28f9b57c6941>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rest_type'] = df['rest_type'].apply(lambda string: string.split(','))


In [10]:
df["reviews_list"] = df["reviews_list"].apply(eval)
reviews_count=[len(r) for r in df['reviews_list']]
df["reviews_list"]=reviews_count

In [11]:
lst = [0]*len(df.index)
df["label"] = lst
for i in df.index:
  if df["rate"][i] < 3.5:
    df["label"][i] = 0
  else:
    df["label"][i] = 1

<ipython-input-11-e56b6aba3cd2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"][i] = 1
<ipython-input-11-e56b6aba3cd2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"][i] = 0


In [12]:
def encode_column(df, column):
  le = preprocessing.LabelEncoder()
  le.fit(df[column].unique())
  df[column] = le.transform(df[column])
  mean = df[column].mean()
  std_dev = df[column].std()
  df[column] = df[column].apply(lambda x: (x - mean) / std_dev)
  return df

In [13]:
# Dropping columns
df = df.drop(["rate", "name"], axis = 1)

In [14]:
# Encoding columns

df = encode_column(df, "online_order")
df = encode_column(df, "book_table")
df = encode_column(df, "location")
df = encode_column(df, "rest_type")
df = encode_column(df, "cuisines")
df = encode_column(df, "listed_in(type)")
df = encode_column(df, "listed_in(city)")

# Running models without sentiment ananlysis

In [33]:
clf_lr = LogisticRegression(random_state = 0, max_iter = 500)
clf_dt_entropy = DecisionTreeClassifier(criterion="entropy", max_depth=15, random_state = 0)
clf_dt_gini = DecisionTreeClassifier(criterion="gini", max_depth=15, random_state = 0)
clf_rf = RandomForestClassifier(max_depth=3, random_state=0)
clf_svc = SVC(random_state = 0)
clf_adaboost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators=100, random_state=0)
clf_mlp_relu = MLPClassifier(random_state=0, hidden_layer_sizes = (256, 128), max_iter=1000, early_stopping = True, activation = "relu", solver = "adam")
clf_mlp_tanh = MLPClassifier(random_state=0, hidden_layer_sizes = (256, 128), max_iter=1000, early_stopping = True, activation = "tanh", solver = "adam")
clf_mlp_sigmoid = MLPClassifier(random_state=0, hidden_layer_sizes = (256, 128), max_iter=1000, early_stopping = True, activation = "logistic", solver = "adam")
clf_mlp_linear = MLPClassifier(random_state=0, hidden_layer_sizes = (256, 128), max_iter=1000, early_stopping = True, activation = "identity", solver = "adam")

In [16]:
def get_k_folds(df, k):
  size = len(df.index) // k
  df_shuffled = df.sample(frac = 1)
  folds = []
  start = 0
  for i in range(k - 1):
    folds.append(df_shuffled.iloc[start:start + size,:])
    start += size
  folds.append(df_shuffled.iloc[start:, :])
  return folds

In [17]:
def train_k_fold(clf, df, k):

  print("===========================================================================")
  print("Model", clf)
  folds = get_k_folds(df, k)
  train_accuracies = []
  test_accuracies = []
  f1_scores = []
  for i in range(k):
    temp = folds.copy()
    test_df = folds[i]
    temp.pop(i)
    train_df = pd.concat(temp)
    train_labels = train_df["label"]
    test_labels = test_df["label"]
    train_df = train_df.drop(["label"], axis = 1)
    test_df = test_df.drop(["label"], axis = 1)
    clf.fit(train_df, train_labels)
    train_predictions = clf.predict(train_df)
    test_predictions = clf.predict(test_df)
    train_loss = log_loss(train_labels, train_predictions)
    test_loss = log_loss(test_labels, test_predictions)
    train_accuracy = accuracy_score(train_labels, train_predictions)
    test_accuracy = accuracy_score(test_labels, test_predictions)
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    macro_f1 = f1_score(test_labels, test_predictions, average="macro")
    f1_scores.append(macro_f1)
    print("===========================================================================")
    print("Training loss for fold {0}: {1}".format(i + 1, train_loss))
    print("Testing loss for fold {0}: {1}".format(i + 1, test_loss))
    print("Training accuracy for fold {0}: {1}".format(i + 1, train_accuracy))
    print("Testing accuracy for fold {0}: {1}".format(i + 1, test_accuracy))
    print("Macro F1-Score for fold {0}: {1}".format(i + 1, macro_f1))
    print("===========================================================================")
  return sum(train_accuracies) / k, sum(test_accuracies) / k, sum(f1_scores) / k


In [18]:
lr_train_accuracy, lr_accuracy, lr_f1 = train_k_fold(clf_lr, df, 4)

Model LogisticRegression(max_iter=500, random_state=0)
Training loss for fold 1: 8.452126213310201
Testing loss for fold 1: 8.385399353364933
Training accuracy for fold 1: 0.7552897884084636
Testing accuracy for fold 1: 0.7572217111315548
Macro F1-Score for fold 1: 0.6855687900295577
Training loss for fold 2: 8.445774716409737
Testing loss for fold 2: 8.379044546263518
Training accuracy for fold 2: 0.7554737810487581
Testing accuracy for fold 2: 0.7574057037718491
Macro F1-Score for fold 2: 0.6854489234888879
Training loss for fold 3: 8.456369126579645
Testing loss for fold 3: 8.706335170971638
Training accuracy for fold 3: 0.7551671266482674
Testing accuracy for fold 3: 0.7479300827966882
Macro F1-Score for fold 3: 0.6744584249881549
Training loss for fold 4: 8.379045232823715
Testing loss for fold 4: 8.423527460373217
Training accuracy for fold 4: 0.7574057037718491
Testing accuracy for fold 4: 0.7561177552897884
Macro F1-Score for fold 4: 0.6875888829168182


In [19]:
dt_entropy_train_accuracy, dt_entropy_accuracy, dt_entropy_f1 = train_k_fold(clf_dt_entropy, df, 4)

Model DecisionTreeClassifier(criterion='entropy', max_depth=15, random_state=0)
Training loss for fold 1: 2.6457999445664178
Testing loss for fold 1: 3.867010939383498
Training accuracy for fold 1: 0.9233977307574364
Testing accuracy for fold 1: 0.8880404783808648
Macro F1-Score for fold 1: 0.8628968665752044
Training loss for fold 2: 2.445611959688487
Testing loss for fold 2: 3.571500823464321
Training accuracy for fold 2: 0.9291934989267095
Testing accuracy for fold 2: 0.8965961361545538
Macro F1-Score for fold 2: 0.8754377003970985
Training loss for fold 3: 2.473154410169382
Testing loss for fold 3: 3.67000537239772
Training accuracy for fold 3: 0.9283961974854339
Testing accuracy for fold 3: 0.8937442502299908
Macro F1-Score for fold 3: 0.8700568472636558
Training loss for fold 4: 2.8131440809572914
Testing loss for fold 4: 3.9845727742990897
Training accuracy for fold 4: 0.9185525912296841
Testing accuracy for fold 4: 0.8846366145354185
Macro F1-Score for fold 4: 0.85989370612948


In [20]:
dt_gini_train_accuracy, dt_gini_accuracy, dt_gini_f1 = train_k_fold(clf_dt_gini, df, 4)

Model DecisionTreeClassifier(max_depth=15, random_state=0)
Training loss for fold 1: 1.888494477914027
Testing loss for fold 1: 3.469820452520671
Training accuracy for fold 1: 0.9453235203925177
Testing accuracy for fold 1: 0.899540018399264
Macro F1-Score for fold 1: 0.8793572321345229
Training loss for fold 2: 1.9901710237365762
Testing loss for fold 2: 3.4857036083730994
Training accuracy for fold 2: 0.9423796381478075
Testing accuracy for fold 2: 0.8990800367985281
Macro F1-Score for fold 2: 0.8795501208797835
Training loss for fold 3: 1.692546905264823
Testing loss for fold 3: 3.2505692723388466
Training accuracy for fold 3: 0.9509966268015946
Testing accuracy for fold 3: 0.9058877644894204
Macro F1-Score for fold 3: 0.8870923696797594
Training loss for fold 4: 1.9287408784761233
Testing loss for fold 4: 3.2918773207185685
Training accuracy for fold 4: 0.9441582336706532
Testing accuracy for fold 4: 0.904691812327507
Macro F1-Score for fold 4: 0.886157188345027


In [21]:
rf_train_accuracy, rf_accuracy, rf_f1 = train_k_fold(clf_rf, df, 4)

Model RandomForestClassifier(max_depth=3, random_state=0)
Training loss for fold 1: 6.128347469534946
Testing loss for fold 1: 5.964176654113605
Training accuracy for fold 1: 0.8225697638761116
Testing accuracy for fold 1: 0.8273229070837167
Macro F1-Score for fold 1: 0.765868085151259
Training loss for fold 2: 6.057383458521893
Testing loss for fold 2: 6.262862371362971
Training accuracy for fold 2: 0.8246243483593989
Testing accuracy for fold 2: 0.8186752529898804
Macro F1-Score for fold 2: 0.7587065333027583
Training loss for fold 3: 6.103990771667163
Testing loss for fold 3: 6.170718146532578
Training accuracy for fold 3: 0.8232750689972401
Testing accuracy for fold 3: 0.8213431462741491
Macro F1-Score for fold 3: 0.7562267353708834
Training loss for fold 4: 6.1124634554822705
Testing loss for fold 4: 6.034083578030345
Training accuracy for fold 4: 0.8230297454768476
Testing accuracy for fold 4: 0.8252989880404784
Macro F1-Score for fold 4: 0.7591380721162581


In [22]:
adaboost_train_accuracy, adaboost_accuracy, adaboost_f1 = train_k_fold(clf_adaboost, df, 4)

Model AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3),
                   n_estimators=100, random_state=0)
Training loss for fold 1: 4.254658071250992
Testing loss for fold 1: 4.8806224229250255
Training accuracy for fold 1: 0.8768169273229071
Testing accuracy for fold 1: 0.8586936522539098
Macro F1-Score for fold 1: 0.8297796332828676
Training loss for fold 2: 4.155097268901521
Testing loss for fold 2: 4.734458034471357
Training accuracy for fold 2: 0.8796994786875192
Testing accuracy for fold 2: 0.8629254829806807
Macro F1-Score for fold 2: 0.8339612467233173
Training loss for fold 3: 4.177337830976114
Testing loss for fold 3: 4.537450113564901
Training accuracy for fold 3: 0.8790555044464888
Testing accuracy for fold 3: 0.8686292548298068
Macro F1-Score for fold 3: 0.8407515482608054
Training loss for fold 4: 4.178399110993633
Testing loss for fold 4: 4.794831166195519
Training accuracy for fold 4: 0.8790248390064398
Testing accuracy for fold 4: 0.861177552897

In [23]:
svc_train_accuracy, svc_accuracy, svc_f1 = train_k_fold(clf_svc, df, 4)

Model SVC(random_state=0)
Training loss for fold 1: 8.856658834950672
Testing loss for fold 1: 8.741210144387411
Training accuracy for fold 1: 0.743575590309721
Testing accuracy for fold 1: 0.746918123275069
Macro F1-Score for fold 1: 0.7214810971942383
Training loss for fold 2: 8.824883769603296
Testing loss for fold 2: 8.83971366348051
Training accuracy for fold 2: 0.7444955535111929
Testing accuracy for fold 2: 0.744066237350506
Macro F1-Score for fold 2: 0.7170646354830491
Training loss for fold 3: 8.920205900644541
Testing loss for fold 3: 8.992228004074198
Training accuracy for fold 3: 0.741735663906777
Testing accuracy for fold 3: 0.7396504139834407
Macro F1-Score for fold 3: 0.7147769923451223
Training loss for fold 4: 8.794168978953152
Testing loss for fold 4: 8.820647991655905
Training accuracy for fold 4: 0.7453848512726158
Testing accuracy for fold 4: 0.7446182152713892
Macro F1-Score for fold 4: 0.7188983998614722


In [34]:
mlp_train_accuracy_relu, mlp_accuracy_relu, mlp_f1_relu = train_k_fold(clf_mlp_relu, df, 4)

Model MLPClassifier(early_stopping=True, hidden_layer_sizes=(256, 128), max_iter=1000,
              random_state=0)
Training loss for fold 1: 6.2173079278728
Testing loss for fold 1: 6.0308922348556235
Training accuracy for fold 1: 0.8199938669119902
Testing accuracy for fold 1: 0.8253909843606255
Macro F1-Score for fold 1: 0.7702292390795725
Training loss for fold 2: 6.041485933945328
Testing loss for fold 2: 6.2533190899276585
Training accuracy for fold 2: 0.8250843299601349
Testing accuracy for fold 2: 0.818951241950322
Macro F1-Score for fold 2: 0.7670714285714286
Training loss for fold 3: 6.097624170442351
Testing loss for fold 3: 6.084913575439237
Training accuracy for fold 3: 0.8234590616375345
Testing accuracy for fold 3: 0.8238270469181233
Macro F1-Score for fold 3: 0.7686916679451969
Training loss for fold 4: 6.101864337471007
Testing loss for fold 4: 6.221551159902337
Training accuracy for fold 4: 0.8233363998773382
Testing accuracy for fold 4: 0.8198712051517939
Macro F1-S

In [25]:
mlp_train_accuracy_tanh, mlp_accuracy_tanh, mlp_f1_tanh = train_k_fold(clf_mlp_tanh, df, 4)

Model MLPClassifier(activation='tanh', early_stopping=True,
              hidden_layer_sizes=(256, 128), max_iter=1000, random_state=0)
Training loss for fold 1: 5.9927644711274395
Testing loss for fold 1: 6.2120136161486785
Training accuracy for fold 1: 0.8264949402023919
Testing accuracy for fold 1: 0.8201471941122355
Macro F1-Score for fold 1: 0.7646353806832786
Training loss for fold 2: 5.898495062621251
Testing loss for fold 2: 6.034065629385178
Training accuracy for fold 2: 0.8292241643667586
Testing accuracy for fold 2: 0.8252989880404784
Macro F1-Score for fold 2: 0.7747085076383671
Training loss for fold 3: 6.059485789551109
Testing loss for fold 3: 5.964162898389644
Training accuracy for fold 3: 0.8245630174793008
Testing accuracy for fold 3: 0.8273229070837167
Macro F1-Score for fold 3: 0.777717784901024
Training loss for fold 4: 6.09338155141299
Testing loss for fold 4: 6.049956288359766
Training accuracy for fold 4: 0.8235817233977307
Testing accuracy for fold 4: 0.8248390

In [26]:
mlp_train_accuracy_sigmoid, mlp_accuracy_sigmoid, mlp_f1_sigmoid = train_k_fold(clf_mlp_sigmoid, df, 4)

Model MLPClassifier(activation='logistic', early_stopping=True,
              hidden_layer_sizes=(256, 128), max_iter=1000, random_state=0)
Training loss for fold 1: 6.139988472238147
Testing loss for fold 1: 6.037249763677823
Training accuracy for fold 1: 0.8222324440355719
Testing accuracy for fold 1: 0.8252069917203312
Macro F1-Score for fold 1: 0.768552387017973
Training loss for fold 2: 5.917564044646811
Testing loss for fold 2: 5.9800538516043185
Training accuracy for fold 2: 0.8286721864458755
Testing accuracy for fold 2: 0.8268629254829807
Macro F1-Score for fold 2: 0.7760436627573359
Training loss for fold 3: 5.9482723374950845
Testing loss for fold 3: 5.970513144769749
Training accuracy for fold 3: 0.8277828886844526
Testing accuracy for fold 3: 0.8271389144434222
Macro F1-Score for fold 3: 0.7824871920003216
Training loss for fold 4: 6.022406261196689
Testing loss for fold 4: 6.2882575456417085
Training accuracy for fold 4: 0.825636307881018
Testing accuracy for fold 4: 0.81

In [27]:
mlp_train_accuracy_linear, mlp_accuracy_linear, mlp_f1_linear = train_k_fold(clf_mlp_linear, df, 4)

Model MLPClassifier(activation='identity', early_stopping=True,
              hidden_layer_sizes=(256, 128), max_iter=1000, random_state=0)
Training loss for fold 1: 6.289329565422321
Testing loss for fold 1: 6.272385644472521
Training accuracy for fold 1: 0.8179086169886538
Testing accuracy for fold 1: 0.8183992640294389
Macro F1-Score for fold 1: 0.7620415468966252
Training loss for fold 2: 6.710849508425088
Testing loss for fold 2: 6.771222370429172
Training accuracy for fold 2: 0.805703771849126
Testing accuracy for fold 2: 0.8039558417663294
Macro F1-Score for fold 2: 0.7622874552419404
Training loss for fold 3: 7.939502360888525
Testing loss for fold 3: 7.931027813552879
Training accuracy for fold 3: 0.770131861392211
Testing accuracy for fold 3: 0.7703771849126035
Macro F1-Score for fold 3: 0.7004471379211619
Training loss for fold 4: 6.281909236342164
Testing loss for fold 4: 6.297796634155813
Training accuracy for fold 4: 0.8181232750689973
Testing accuracy for fold 4: 0.81766

In [28]:
print("============================================================")
print("Combined results")
print("============================================================")
print("Training accuracy for Logisitic regression:", lr_train_accuracy)
print("Testing accuracy for Logisitic regression:", lr_accuracy)
print("Macro_f1 score for Logisitic regression: ", lr_f1)
print("============================================================")
print("Training accuracy for Decision tree (entropy):", dt_entropy_train_accuracy)
print("Testing accuracy for Decision tree (entropy):", dt_entropy_accuracy)
print("Macro_f1 score for Decision tree (entropy): ", dt_entropy_f1)
print("============================================================")
print("Training accuracy for Decision tree (gini):", dt_gini_train_accuracy)
print("Testing accuracy for Decision tree (gini):", dt_gini_accuracy)
print("Macro_f1 score for Decision tree (gini): ", dt_gini_f1)
print("============================================================")
print("Training accuracy for Random Forest:", rf_train_accuracy)
print("Testing accuracy for Random Forest:", rf_accuracy)
print("Macro_f1 score for Random forest: ", rf_f1)
print("============================================================")
print("Training accuracy for AdaBoost classifier:", adaboost_train_accuracy)
print("Testing accuracy for AdaBoost classifier:", adaboost_accuracy)
print("Macro_f1 score for AdaBoost classifier: ", adaboost_f1)
print("============================================================")
print("Training accuracy for Support vector classifier:", svc_train_accuracy)
print("Testing accuracy for Support vector classifier:", svc_accuracy)
print("Macro_f1 score for Support vector classifier: ", svc_f1)
print("============================================================")
print("Training Accuracy for MLP classifier (relu):", mlp_train_accuracy_relu)
print("Testing accuracy for MLP classifier (relu):", mlp_accuracy_relu)
print("Macro_f1 score for MLP classifier (relu): ", mlp_f1_relu)
print("============================================================")
print("Training Accuracy for MLP classifier (tanh):", mlp_train_accuracy_tanh)
print("Testing accuracy for MLP classifier (tanh):", mlp_accuracy_tanh)
print("Macro_f1 score for MLP classifier (tanh): ", mlp_f1_tanh)
print("============================================================")
print("Training Accuracy for MLP classifier (sigmoid):", mlp_train_accuracy_sigmoid)
print("Testing accuracy for MLP classifier (sigmoid):", mlp_accuracy_sigmoid)
print("Macro_f1 score for MLP classifier (sigmoid): ", mlp_f1_sigmoid)
print("============================================================")
print("Training Accuracy for MLP classifier (linear):", mlp_train_accuracy_linear)
print("Testing accuracy for MLP classifier (linear):", mlp_accuracy_linear)
print("Macro_f1 score for MLP classifier (linear): ", mlp_f1_linear)
print("============================================================")

Combined results
Training accuracy for Logisitic regression: 0.7558340999693345
Testing accuracy for Logisitic regression: 0.7546688132474701
Macro_f1 score for Logisitic regression:  0.6832662553558547
Training accuracy for Decision tree (entropy): 0.924885004599816
Testing accuracy for Decision tree (entropy): 0.890754369825207
Macro_f1 score for Decision tree (entropy):  0.8670712800913596
Training accuracy for Decision tree (gini): 0.9457145047531432
Testing accuracy for Decision tree (gini): 0.90229990800368
Macro_f1 score for Decision tree (gini):  0.8830392277597732
Training accuracy for Random Forest: 0.8233747316773996
Testing accuracy for Random Forest: 0.8231600735970561
Macro_f1 score for Random forest:  0.7599848564852896
Training accuracy for AdaBoost classifier: 0.8786491873658386
Testing accuracy for AdaBoost classifier: 0.8628564857405704
Macro_f1 score for AdaBoost classifier:  0.8340015716368202
Training accuracy for Support vector classifier: 0.7437979147500766
Test

# Doing sentiment analysis

In [ ]:
url = "/content/drive/MyDrive/zomato.csv"
df_new = pd.read_csv(url)

In [ ]:
df_new=df_new.drop(['dish_liked'],axis=1)
df_new=df_new.drop(['url', 'address', 'phone', 'menu_item', 'name'], axis=1)
df_new=df_new.dropna()
df_new=df_new.drop_duplicates()

df_new.rate=df_new.rate.replace({'-':'0/5','NEW':'0/5'})
df_new.rate=df_new.rate.astype(str)
df_new.rate=df_new.rate.apply(lambda row: row.replace('/5',''))
df_new.rate=df_new.rate.astype(float)

df_new['approx_cost(for two people)']=df_new['approx_cost(for two people)'].astype(str)
df_new['approx_cost(for two people)']=df_new['approx_cost(for two people)'].apply(lambda row: row.replace(',',''))
df_new['approx_cost(for two people)']=df_new['approx_cost(for two people)'].astype(float)

df_new['rest_type'] = df_new['rest_type'].astype('str')
df_new['rest_type'] = df_new['rest_type'].apply(lambda string: string.split(','))
df_new['rest_type'] = df_new['rest_type'].apply(lambda lst: sorted([string.strip() for string in lst]))
df_new['rest_type'] = df_new['rest_type'].apply(lambda lst: ', '.join(lst))

# correct cuisines column (change order)
df_new['cuisines'] = df_new['cuisines'].astype('str')
df_new['cuisines'] = df_new['cuisines'].apply(lambda string: string.split(','))
df_new['cuisines'] = df_new['cuisines'].apply(lambda lst: sorted([string.strip() for string in lst]))
df_new['cuisines'] = df_new['cuisines'].apply(lambda lst: ', '.join(lst))


df_new = encode_column(df_new, "online_order")
df_new = encode_column(df_new, "book_table")
df_new = encode_column(df_new, "location")
df_new = encode_column(df_new, "rest_type")
df_new = encode_column(df_new, "cuisines")
df_new = encode_column(df_new, "listed_in(type)")
df_new = encode_column(df_new, "listed_in(city)")


lst = [0]*len(df_new.index)
df_new["label"] = lst
for i in df_new.index:
  if df_new["rate"][i] < 3.5:
    df_new["label"][i] = 0
  else:
    df_new["label"][i] = 1

df_new=df_new.drop(['rate'], axis=1)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-117-822c729dbab7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['approx_cost(for two people)']=df_new['approx_cost(for two people)'].astype(str)
<ipython-input-117-822c729dbab7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [ ]:
df_new["reviews_list"] = df_new["reviews_list"].apply(eval)

In [ ]:
# Giving sentiment scores to reviews using vader
sentiment_analyzer = SentimentIntensityAnalyzer()
df_new["review_sentiment"] = [0.0]*len(df_new.index)

for i in df_new.index:
  pol_sum = 0
  count = 0
  for j in df_new["reviews_list"][i]:
    sentence = j[1][9:]
    pol_sum += sentiment_analyzer.polarity_scores(sentence)['compound']
    count += 1
  if count != 0:
    df_new["review_sentiment"][i] = pol_sum / count

df_new = df_new.drop(["reviews_list"], axis = 1)


<ipython-input-119-4b47059f0046>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["review_sentiment"][i] = pol_sum / count


In [ ]:
lr_train_accuracy_sent, lr_accuracy_sent, lr_f1_sent = train_k_fold(clf_lr, df_new, 4)

Model LogisticRegression(max_iter=500, random_state=0)
Training loss for fold 1: 7.147268337501046
Testing loss for fold 1: 7.293680530729923
Training accuracy for fold 1: 0.7930685477687471
Testing accuracy for fold 1: 0.7888295914611704
Macro F1-Score for fold 1: 0.7374159098898467
Training loss for fold 2: 7.125022574585393
Testing loss for fold 2: 7.2873233769986685
Training accuracy for fold 2: 0.7937126207636865
Testing accuracy for fold 2: 0.7890136179609863
Macro F1-Score for fold 2: 0.7383317617167859
Training loss for fold 3: 7.286040245744612
Testing loss for fold 3: 7.096637826616509
Training accuracy for fold 3: 0.7890507590860297
Testing accuracy for fold 3: 0.7945344129554656
Macro F1-Score for fold 3: 0.7431499773035893
Training loss for fold 4: 7.18668378300735
Testing loss for fold 4: 7.140472644402886
Training accuracy for fold 4: 0.7919273708747393
Testing accuracy for fold 4: 0.7932652497929892
Macro F1-Score for fold 4: 0.7434355417049086


In [ ]:
dt_entropy_train_accuracy_sent, dt_entropy_accuracy_sent, dt_entropy_f1_sent = train_k_fold(clf_dt_entropy, df_new, 4)

Model DecisionTreeClassifier(criterion='entropy', max_depth=15, random_state=0)
Training loss for fold 1: 1.860173216326052
Testing loss for fold 1: 3.028698792834107
Training accuracy for fold 1: 0.9461432295660175
Testing accuracy for fold 1: 0.9123113728376886
Macro F1-Score for fold 1: 0.8970277271518179
Training loss for fold 2: 1.906786671261571
Testing loss for fold 2: 2.9873837312672924
Training accuracy for fold 2: 0.9447937432909063
Testing accuracy for fold 2: 0.9135075450864925
Macro F1-Score for fold 2: 0.8974723360814094
Training loss for fold 3: 1.9936521835743004
Testing loss for fold 3: 3.2511666539153046
Training accuracy for fold 3: 0.942278791596381
Testing accuracy for fold 3: 0.9058704453441295
Macro F1-Score for fold 3: 0.8883711823822289
Training loss for fold 4: 1.8676500760830799
Testing loss for fold 4: 3.158712621485164
Training accuracy for fold 4: 0.9459268801374064
Testing accuracy for fold 4: 0.9085472444567118
Macro F1-Score for fold 4: 0.89070998241047

In [ ]:
dt_gini_train_accuracy_sent, dt_gini_accuracy_sent, dt_gini_f1_sent = train_k_fold(clf_dt_gini, df_new, 4)

Model DecisionTreeClassifier(max_depth=15, random_state=0)
Training loss for fold 1: 1.074157183701555
Testing loss for fold 1: 2.5392752813371446
Training accuracy for fold 1: 0.9689004753872106
Testing accuracy for fold 1: 0.9264814133235186
Macro F1-Score for fold 1: 0.9144956938965196
Training loss for fold 2: 1.4851749400646548
Testing loss for fold 2: 2.755386871569786
Training accuracy for fold 2: 0.9570004600521392
Testing accuracy for fold 2: 0.9202245123297755
Macro F1-Score for fold 2: 0.9045594967091835
Training loss for fold 3: 1.72140206831927
Testing loss for fold 3: 2.9905599169922796
Training accuracy for fold 3: 0.9501610182487349
Testing accuracy for fold 3: 0.9134155318365844
Macro F1-Score for fold 3: 0.8984593369667316
Training loss for fold 4: 1.599631895791814
Testing loss for fold 4: 2.7932685089775626
Training accuracy for fold 4: 0.95368666421298
Testing accuracy for fold 4: 0.9191277946453216
Macro F1-Score for fold 4: 0.9036525811582594


In [ ]:
rf_train_accuracy_sent, rf_accuracy_sent, rf_f1_sent = train_k_fold(clf_rf, df_new, 4)

Model RandomForestClassifier(max_depth=3, random_state=0)
Training loss for fold 1: 5.720383801838915
Testing loss for fold 1: 5.809545106649784
Training accuracy for fold 1: 0.8343812298727189
Testing accuracy for fold 1: 0.8317997791682002
Macro F1-Score for fold 1: 0.7751803868462306
Training loss for fold 2: 5.808309751442756
Testing loss for fold 2: 5.958917949818989
Training accuracy for fold 2: 0.8318356080355773
Testing accuracy for fold 2: 0.8274751564225249
Macro F1-Score for fold 2: 0.7678740009483248
Training loss for fold 3: 5.9206005973570015
Testing loss for fold 3: 5.971630344368986
Training accuracy for fold 3: 0.828584572918264
Testing accuracy for fold 3: 0.8271071034228928
Macro F1-Score for fold 3: 0.771727671554782
Training loss for fold 4: 5.858277350322376
Testing loss for fold 4: 5.713679135560266
Training accuracy for fold 4: 0.8303889093362777
Testing accuracy for fold 4: 0.8345753979206919
Macro F1-Score for fold 4: 0.7771054042278486


In [ ]:
adaboost_train_accuracy_sent, adaboost_accuracy_sent, adaboost_f1_sent = train_k_fold(clf_adaboost, df_new, 4)

Model AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3),
                   n_estimators=100, random_state=0)
Training loss for fold 1: 3.41208380204526
Testing loss for fold 1: 4.141023604249134
Training accuracy for fold 1: 0.9012114706333384
Testing accuracy for fold 1: 0.8801067353698933
Macro F1-Score for fold 1: 0.8575463480486074
Training loss for fold 2: 3.4512786592686306
Testing loss for fold 2: 3.9153830154001845
Training accuracy for fold 2: 0.9000766753565405
Testing accuracy for fold 2: 0.8866396761133604
Macro F1-Score for fold 2: 0.8650850525731324
Training loss for fold 3: 3.519075968402658
Testing loss for fold 3: 4.334889239754863
Training accuracy for fold 3: 0.898113786229106
Testing accuracy for fold 3: 0.8744939271255061
Macro F1-Score for fold 3: 0.8504988517701804
Training loss for fold 4: 3.406893756922477
Testing loss for fold 4: 3.930911224082387
Training accuracy for fold 4: 0.9013617960986382
Testing accuracy for fold 4: 0.8861900818842

In [ ]:
svc_train_accuracy_sent, svc_accuracy_sent, svc_f1_sent = train_k_fold(clf_svc, df_new, 4)

Model SVC(random_state=0)
Training loss for fold 1: 8.912039154944235
Testing loss for fold 1: 8.815915174504037
Training accuracy for fold 1: 0.7419720901702193
Testing accuracy for fold 1: 0.7447552447552448
Macro F1-Score for fold 1: 0.719586061031688
Training loss for fold 2: 8.845304122383864
Testing loss for fold 2: 8.965281101758775
Training accuracy for fold 2: 0.7439043091550376
Testing accuracy for fold 2: 0.7404306220095693
Macro F1-Score for fold 2: 0.7132681755826376
Training loss for fold 3: 8.962886535962461
Testing loss for fold 3: 9.057442504289014
Training accuracy for fold 3: 0.7404999233246434
Testing accuracy for fold 3: 0.7377622377622378
Macro F1-Score for fold 3: 0.7132420375407795
Training loss for fold 4: 8.895365752053522
Testing loss for fold 4: 8.754725013955385
Training accuracy for fold 4: 0.7424549135075451
Testing accuracy for fold 4: 0.7465268193946085
Macro F1-Score for fold 4: 0.7222392874417707


In [ ]:
mlp_train_accuracy_relu_sent, mlp_accuracy_relu_sent, mlp_f1_relu_sent = train_k_fold(clf_mlp_relu, df_new, 4)

Model MLPClassifier(early_stopping=True, hidden_layer_sizes=(256, 128), max_iter=1000,
              random_state=0)


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 1: 5.372902280716826
Testing loss for fold 1: 5.266069565467581
Training accuracy for fold 1: 0.8444410366508204
Testing accuracy for fold 1: 0.847534044902466
Macro F1-Score for fold 1: 0.8119239770517767


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 2: 5.4873032352794855
Testing loss for fold 2: 5.590229225592935
Training accuracy for fold 2: 0.8411286612482748
Testing accuracy for fold 2: 0.8381486934118513
Macro F1-Score for fold 2: 0.8037325030353897


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 3: 5.474600255366781
Testing loss for fold 3: 5.507608299154077
Training accuracy for fold 3: 0.8414967029596687
Testing accuracy for fold 3: 0.840541037909459
Macro F1-Score for fold 3: 0.8006212588189731


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 4: 5.418622542809396
Testing loss for fold 4: 5.376815563288177
Training accuracy for fold 4: 0.8431174089068826
Testing accuracy for fold 4: 0.8443279050510627
Macro F1-Score for fold 4: 0.8061993535463899


In [ ]:
mlp_train_accuracy_sigmoid_sent, mlp_accuracy_sigmoid_sent, mlp_f1_sigmoid_sent = train_k_fold(clf_mlp_sigmoid, df_new, 4)

Model MLPClassifier(activation='logistic', early_stopping=True,
              hidden_layer_sizes=(256, 128), max_iter=1000, random_state=0)


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 1: 5.180099129109911
Testing loss for fold 1: 4.960966063235892
Training accuracy for fold 1: 0.8500230026069622
Testing accuracy for fold 1: 0.8563673168936327
Macro F1-Score for fold 1: 0.8257599909799578


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 2: 5.14090576783634
Testing loss for fold 2: 5.437684463965117
Training accuracy for fold 2: 0.8511577978837601
Testing accuracy for fold 2: 0.8425653294074347
Macro F1-Score for fold 2: 0.8088736891262536


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 3: 5.24153246569007
Testing loss for fold 3: 5.33915663743529
Training accuracy for fold 3: 0.8482441343352246
Testing accuracy for fold 3: 0.8454177401545823
Macro F1-Score for fold 3: 0.815798951708241


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 4: 5.298906975112747
Testing loss for fold 4: 5.135293829078739
Training accuracy for fold 4: 0.8465832413200834
Testing accuracy for fold 4: 0.85132026865397
Macro F1-Score for fold 4: 0.8190326844147018


In [ ]:
mlp_train_accuracy_tanh_sent, mlp_accuracy_tanh_sent, mlp_f1_tanh_sent = train_k_fold(clf_mlp_tanh, df_new, 4)

Model MLPClassifier(activation='tanh', early_stopping=True,
              hidden_layer_sizes=(256, 128), max_iter=1000, random_state=0)


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 1: 5.298741710218181
Testing loss for fold 1: 5.351869767720869
Training accuracy for fold 1: 0.8465879466339519
Testing accuracy for fold 1: 0.8450496871549503
Macro F1-Score for fold 1: 0.8127457431061991


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 2: 5.434347437752785
Testing loss for fold 2: 5.243828760889741
Training accuracy for fold 2: 0.8426621683790829
Testing accuracy for fold 2: 0.8481781376518218
Macro F1-Score for fold 2: 0.8113857769548722


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 3: 5.170570520844417
Testing loss for fold 3: 5.396373154649768
Training accuracy for fold 3: 0.8502990338905075
Testing accuracy for fold 3: 0.8437615016562385
Macro F1-Score for fold 3: 0.8061334312723556


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 4: 5.12622393062477
Testing loss for fold 4: 5.214731775463905
Training accuracy for fold 4: 0.8515826278984173
Testing accuracy for fold 4: 0.8490201490477505
Macro F1-Score for fold 4: 0.820600782219953


In [ ]:
mlp_train_accuracy_linear_sent, mlp_accuracy_linear_sent, mlp_f1_linear_sent = train_k_fold(clf_mlp_linear, df_new, 4)

Model MLPClassifier(activation='identity', early_stopping=True,
              hidden_layer_sizes=(256, 128), max_iter=1000, random_state=0)


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 1: 7.098512304449313
Testing loss for fold 1: 7.023518161562531
Training accuracy for fold 1: 0.7944793743290907
Testing accuracy for fold 1: 0.7966507177033493
Macro F1-Score for fold 1: 0.7627152160151549


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 2: 6.724622360396409
Testing loss for fold 2: 6.998143623202767
Training accuracy for fold 2: 0.8053059346725963
Testing accuracy for fold 2: 0.7973868237026132
Macro F1-Score for fold 2: 0.7316830237825671


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 3: 5.8856291455201974
Testing loss for fold 3: 5.914413312139601
Training accuracy for fold 3: 0.8295966876245975
Testing accuracy for fold 3: 0.8287633419212367
Macro F1-Score for fold 3: 0.7770284470095146


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifi

Training loss for fold 4: 6.746000583749859
Testing loss for fold 4: 6.4064151468979915
Training accuracy for fold 4: 0.8046865415286468
Testing accuracy for fold 4: 0.8145183549544577
Macro F1-Score for fold 4: 0.7641462699082646


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


In [ ]:
print("============================================================")
print("Combined results")
print("============================================================")
print("Training accuracy for Logisitic regression:", lr_train_accuracy_sent)
print("Testing accuracy for Logisitic regression:", lr_accuracy_sent)
print("Macro f1 score for Logisitic regression: ", lr_f1_sent)
print("============================================================")
print("Training accuracy for Decision tree (entropy):", dt_entropy_train_accuracy_sent)
print("Testing accuracy for Decision tree (entropy):", dt_entropy_accuracy_sent)
print("Macro f1 score for Decision tree (entropy): ", dt_entropy_f1_sent)
print("============================================================")
print("Training accuracy for Decision tree (gini):", dt_gini_train_accuracy_sent)
print("Testing accuracy for Decision tree (gini):", dt_gini_accuracy_sent)
print("Macro f1 score for Decision tree (gini): ", dt_gini_f1_sent)
print("============================================================")
print("Training accuracy for Random Forest:", rf_train_accuracy_sent)
print("Testing accuracy for Random Forest:", rf_accuracy_sent)
print("Macro f1 score for Random forest: ", rf_f1_sent)
print("============================================================")
print("Training accuracy for AdaBoost classifier:", adaboost_train_accuracy_sent)
print("Testing accuracy for AdaBoost classifier:", adaboost_accuracy_sent)
print("Macro f1 score for AdaBoost classifier: ", adaboost_f1_sent)
print("============================================================")
print("Training accuracy for Support vector classifier:", svc_train_accuracy_sent)
print("Testing accuracy for Support vector classifier:", svc_accuracy_sent)
print("Macro f1 score for Support vector classifier: ", svc_f1_sent)
print("============================================================")
print("Training Accuracy for MLP classifier (relu):", mlp_train_accuracy_relu_sent)
print("Testing accuracy for MLP classifier (relu):", mlp_accuracy_relu_sent)
print("Macro f1 score for MLP classifier (relu): ", mlp_f1_relu_sent)
print("============================================================")
print("Training Accuracy for MLP classifier (tanh):", mlp_train_accuracy_tanh_sent)
print("Testing accuracy for MLP classifier (tanh):", mlp_accuracy_tanh_sent)
print("Macro f1 score for MLP classifier (tanh): ", mlp_f1_tanh_sent)
print("============================================================")
print("Training Accuracy for MLP classifier (sigmoid):", mlp_train_accuracy_sigmoid_sent)
print("Testing accuracy for MLP classifier (sigmoid):", mlp_accuracy_sigmoid_sent)
print("Macro f1 score for MLP classifier (sigmoid): ", mlp_f1_sigmoid_sent)
print("============================================================")
print("Training Accuracy for MLP classifier (linear):", mlp_train_accuracy_linear_sent)
print("Testing accuracy for MLP classifier (linear):", mlp_accuracy_linear_sent)
print("Macro f1 score for MLP classifier (linear): ", mlp_f1_linear_sent)
print("============================================================")

Combined results
Training accuracy for Logisitic regression: 0.7921621972675654
Testing accuracy for Logisitic regression: 0.7921469573760493
Macro f1 score for Logisitic regression:  0.7414123759153592
Training accuracy for Decision tree (entropy): 0.9480750889734214
Testing accuracy for Decision tree (entropy): 0.9117153756146958
Macro f1 score for Decision tree (entropy):  0.8951758112226077
Training accuracy for Decision tree (gini): 0.9574371544752662
Testing accuracy for Decision tree (gini): 0.9198123130338001
Macro f1 score for Decision tree (gini):  0.9052917771826735
Training accuracy for Random Forest: 0.8305921796949772
Testing accuracy for Random Forest: 0.8296643737767334
Macro f1 score for Random forest:  0.7722544261081707
Training accuracy for AdaBoost classifier: 0.9001909320794057
Testing accuracy for AdaBoost classifier: 0.8818576051232544
Macro f1 score for AdaBoost classifier:  0.8589565934619019
Training accuracy for Support vector classifier: 0.7422078090393613
